In [1]:
import requests


url = 'https://cricsheet.org/downloads/ipl_json.zip'
r = requests.get(url, allow_redirects=True)

open('ipl.zip', 'wb').write(r.content)

3616233

In [1]:
from zipfile import ZipFile

In [7]:
with ZipFile('ipl.zip','r') as zobj:
    zobj.extractall(path='ipl')

In [8]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [9]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ----------------------- ---------------- 6.8/11.5 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 31.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----------------------------- ---------- 9.2/12.6 MB 43.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 36.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd

In [13]:
from glob import glob

In [19]:
df = pd.concat([pd.read_json(f_name) for f_name in glob('ipl\\*')])

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
df=[]
for fname in glob('ipl\\*.json'):
    #df1=pd.read_json(fname)
    #df=df.concat(df1)
    print(fname)

In [34]:
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [59]:
import json
import pandas as pd
from glob import glob
dflist=[]
for fname in glob('ipl\\*.json'):
    #print(fname)
    with open(fname) as jsonfile:
        data1 = json.load(jsonfile)
    data = data1['info']
    if 'result' in data['outcome']:
        continue
    pddf={
    "match_id":fname[4:fname.index('.')],    
    "venue":data['venue'],
    "match_date":data['dates'][0],
    "winner":data['outcome']['winner'],
    "pom":data['player_of_match'][0],
    "season":data['season'],
    "team1":data['teams'][0],
    "team2":data['teams'][1],
    "toss_winner":data['toss']['winner'],
    "toss_dec":data['toss']['decision']
    }
    dflist.append(pddf)
ipl_sum=pd.DataFrame(dflist)
ipl_sum.head()

,match_id,venue,match_date,winner,pom,season,team1,team2,toss_winner,toss_dec
0,1082591,"Rajiv Gandhi International Stadium, Uppal",2017-04-05,Sunrisers Hyderabad,Yuvraj Singh,2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field
1,1082592,Maharashtra Cricket Association Stadium,2017-04-06,Rising Pune Supergiant,SPD Smith,2017,Rising Pune Supergiant,Mumbai Indians,Rising Pune Supergiant,field
2,1082593,Saurashtra Cricket Association Stadium,2017-04-07,Kolkata Knight Riders,CA Lynn,2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field
3,1082594,Holkar Cricket Stadium,2017-04-08,Kings XI Punjab,GJ Maxwell,2017,Kings XI Punjab,Rising Pune Supergiant,Kings XI Punjab,field
4,1082595,M.Chinnaswamy Stadium,2017-04-08,Royal Challengers Bangalore,KM Jadhav,2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat


In [130]:
dflist=[]
pddf={}
data=[]
data1=[]
i=0
for fname in glob('ipl\\*.json'):
    #print(fname)
    with open(fname) as jsonfile:
        data1 = json.load(jsonfile)
    if 'innings' not in data1:
        continue
    datai = data1['innings']
    #print(len(data))
    for i in range(len(datai)): #2 innings
        if 'overs' not in datai[i]:
            continue
        datao = datai[i]['overs']
        
        for j in range(len(datao)): #20 overs
            if 'deliveries' not in datao[j]:
                continue
            datad = datao[j]['deliveries']
            for k in range(len(datad)): #6 deliveries
                pddf={
                "match_id":fname[4:fname.index('.')],    
                "innings":i+1,
                "team":datai[i]['team'],
                "over":datao[j]['over'],
                "ball":k,
                "batter":datad[k]['batter'],
                "non_striker":datad[k]['non_striker'],
                "bowler":datad[k]['bowler'],
                "runs_batter":datad[k]['runs']['batter'],
                "runs_extras":datad[k]['runs']['extras'],
                "runs_total":datad[k]['runs']['total']
                }
                dflist.append(pddf)
tmp=pd.DataFrame(dflist)
tmp.head()


,match_id,innings,team,over,ball,batter,non_striker,bowler,runs_batter,runs_extras,runs_total
0,1082591,1,Sunrisers Hyderabad,0,0,DA Warner,S Dhawan,TS Mills,0,0,0
1,1082591,1,Sunrisers Hyderabad,0,1,DA Warner,S Dhawan,TS Mills,0,0,0
2,1082591,1,Sunrisers Hyderabad,0,2,DA Warner,S Dhawan,TS Mills,4,0,4
3,1082591,1,Sunrisers Hyderabad,0,3,DA Warner,S Dhawan,TS Mills,0,0,0
4,1082591,1,Sunrisers Hyderabad,0,4,DA Warner,S Dhawan,TS Mills,0,2,2


In [ ]:
dflist=[]
pddf={}
data=[]
data1=[]
i=0
for fname in glob('ipl\\*.json'):
    #print(fname)
    with open(fname) as jsonfile:
        data1 = json.load(jsonfile)
    if 'innings' not in data1:
        continue
    datai = data1['innings']
    #print(len(data))
    for i in range(len(datai)): #2 innings
        if 'overs' not in datai[i]:
            continue
        datao = datai[i]['overs']
        
        for j in range(len(datao)): #20 overs
            if 'deliveries' not in datao[j]:
                continue
            datad = datao[j]['deliveries']
            for k in range(len(datad)): #6 deliveries
                if 'wickets' not in datad:
                    continue 
                pddf={
                "match_id":fname[4:fname.index('.')],    
                "innings":i+1,
                #"team":datai[i]['team'],
                "over":datao[j]['over'],
                "ball":k,
                #"batter":datad[k]['batter'],
                #"non_striker":datad[k]['non_striker'],
                "batsman":datad[k]['wickets']['player_out'],
                "runs_batter":datad[k]['runs']['batter'],
                "runs_extras":datad[k]['runs']['extras'],
                "runs_total":datad[k]['runs']['total']
                }
                dflist.append(pddf)
tmp=pd.DataFrame(dflist)
tmp.head()

